In [44]:
from bs4 import BeautifulSoup
from time import sleep
import requests
import pandas as pd

In [45]:
# 変数urlにSUUMOホームページのURLを格納する。東京都／目黒区 15万円以上～20万円以下 2LDK／3LDKのURLを格納
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13110&ta=13&cb=15.0&ct=20.0&md=07&md=10&et=9999999&mb=0&mt=9999999&cn=9999999&fw2=&?page={}'

for i in range(1, 4):
    # 変数target_urlに、アクセス先のURLを格納する
    target_url = url.format(i)
    
    # target_urlへのアクセス結果を、変数rに格納
    r = requests.get(target_url)

    # print()してtarget_urlを確認する
    print(target_url)
    
    sleep(1)
    
    # 取得結果を解析してsoupに格納
    soup = BeautifulSoup(r.text)

https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13110&ta=13&cb=15.0&ct=20.0&md=07&md=10&et=9999999&mb=0&mt=9999999&cn=9999999&fw2=&?page=1
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13110&ta=13&cb=15.0&ct=20.0&md=07&md=10&et=9999999&mb=0&mt=9999999&cn=9999999&fw2=&?page=2
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13110&ta=13&cb=15.0&ct=20.0&md=07&md=10&et=9999999&mb=0&mt=9999999&cn=9999999&fw2=&?page=3


In [46]:
d_list = []
# すべての物件情報を取得する
contents = soup.find_all('div', class_='cassetteitem')

# 各物件情報をforループで取得する
for content in contents:
    # 物件情報と部屋情報を取得しておく
    detail = content.find('div', class_='cassetteitem_content')
    table = content.find('table', class_='cassetteitem_other')
    
    # 物件情報から必要な情報を取得する
    title = detail.find('div', class_='cassetteitem_content-title').text
    address = detail.find('li', class_='cassetteitem_detail-col1').text
    access = detail.find('li', class_='cassetteitem_detail-col2').text
    age = detail.find('li', class_='cassetteitem_detail-col3').text

    # 部屋情報のブロックから、各部屋情報を取得する
    tr_tags = table.find_all('tr', class_='js-cassette_link')
    
    # 各部屋情報をforループで取得する
    for tr_tag in tr_tags:        
        
        # 部屋情報の行から、欲しい情報を取得する
        floor, price, first_fee, capacity = tr_tag.find_all('td')[2:6]
        
        # さらに細かい情報を取得する
        fee, management_fee = price.find_all('li')
        deposit, gratuity = first_fee.find_all('li')
        madori, menseki = capacity.find_all('li')

        # 取得したすべての情報を辞書に格納する
        d = {
            '物件名': title,
            '住所': address,
            'アクセス': access,
            '築年数': age,
            '階数': floor.text,
            '賃料': fee.text,
            '管理費': management_fee.text,
            '敷金': deposit.text,
            '礼金': gratuity.text,
            '間取り': madori.text,
            '面積': menseki.text
        }
        
        # 取得した辞書をd_listに格納する
        d_list.append(d)

In [47]:
d = pd.DataFrame(d_list)

In [48]:
d

,物件名,住所,アクセス,築年数,階数,賃料,管理費,敷金,礼金,間取り,面積
0,エルファーロ大岡山II,東京都目黒区大岡山１,\n東急目黒線/大岡山駅 歩7分\n東急大井町線/緑が丘駅 歩14分\n東急目黒線/洗足駅 ...,\n築1年\n4階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t3階,19.7万円,10000円,-,-,2LDK,45.02m2
1,ウェルスクエア学芸大学,東京都目黒区鷹番３,\n東急東横線/学芸大学駅 歩2分\n東急東横線/祐天寺駅 歩17分\n東急東横線/都立大学...,\n新築\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,18.6万円,12000円,18.6万円,18.6万円,2LDK,40.8m2
2,ウェルスクエア学芸大学,東京都目黒区鷹番３,\n東急東横線/学芸大学駅 歩2分\n東急東横線/祐天寺駅 歩17分\n東急東横線/都立大学...,\n新築\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,19.4万円,12000円,19.4万円,19.4万円,2LDK,40.8m2
3,ウェルスクエア学芸大学,東京都目黒区鷹番３,\n東急東横線/学芸大学駅 歩2分\n東急東横線/祐天寺駅 歩17分\n東急東横線/都立大学...,\n新築\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t3階,19.6万円,12000円,19.6万円,19.6万円,2LDK,40.8m2
4,ウェルスクエア学芸大学,東京都目黒区鷹番３,\n東急東横線/学芸大学駅 歩2分\n東急東横線/祐天寺駅 歩17分\n東急東横線/都立大学...,\n新築\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,19.7万円,12000円,19.7万円,19.7万円,2LDK,40.84m2
5,ウェルスクエア学芸大学,東京都目黒区鷹番３,\n東急東横線/学芸大学駅 歩2分\n東急東横線/祐天寺駅 歩17分\n東急東横線/都立大学...,\n新築\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,19.8万円,12000円,19.8万円,19.8万円,2LDK,40.8m2
6,ウェルスクエア学芸大学,東京都目黒区鷹番３,\n東急東横線/学芸大学駅 歩2分\n東急東横線/祐天寺駅 歩17分\n東急東横線/都立大学...,\n新築\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t3階,19.9万円,12000円,19.9万円,19.9万円,2LDK,40.84m2
7,セザール大岡山ガーデン,東京都目黒区大岡山１,\n東急目黒線/大岡山駅 歩10分\n東急大井町線/大岡山駅 歩10分\n東急大井町線/緑が...,\n築28年\n4階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,15万円,10000円,15万円,15万円,3LDK,53.91m2
8,東急目黒線 大岡山駅 4階建 築28年,東京都目黒区大岡山１,\n東急目黒線/大岡山駅 歩10分\n東急大井町線/大岡山駅 歩10分\n東急大井町線/緑が...,\n築28年\n4階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,15万円,10000円,15万円,15万円,3LDK,53.91m2
9,東急東横線 学芸大学駅 3階建 築31年,東京都目黒区目黒本町１,\n東急東横線/学芸大学駅 歩12分\n東急目黒線/武蔵小山駅 歩14分\n東急目黒線/西小...,\n築31年\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,15.8万円,-,15.8万円,15.8万円,2LDK,53.12m2


In [49]:
#アクセスから、「路線名/」を削除
d['アクセス'] = d['アクセス'].apply(lambda x: x.replace('東急大井町線/', ''))
d['アクセス'] = d['アクセス'].apply(lambda x: x.replace('東急目黒線/', ''))
d['アクセス'] = d['アクセス'].apply(lambda x: x.replace('東急東横線/', ''))
d['アクセス'] = d['アクセス'].apply(lambda x: x.replace('東京メトロ日比谷線/', ''))
d['アクセス'] = d['アクセス'].apply(lambda x: x.replace('ＪＲ山手線/', ''))
d['アクセス'] = d['アクセス'].apply(lambda x: x.replace('東急田園都市線/', ''))
d['アクセス'] = d['アクセス'].apply(lambda x: x.replace('京王井の頭線/', ''))

#築年数で「築1年」を「新築」に変更
d['築年数'] = d['築年数'].apply(lambda x: x.replace('築1年', '新築'))

#階数から、「階」を削除
d['階数'] = d['階数'].apply(lambda x: x.replace('階', ''))

In [50]:
# 重複した行を特定する条件を指定
duplicate_condition = d.duplicated(subset=['住所', '築年数', '階数'], keep=False)

# 重複した行を抽出
duplicates = d[duplicate_condition]

# 重複した行を表示
print(duplicates)

                       物件名           住所  \
0              エルファーロ大岡山II   東京都目黒区大岡山１   
1              ウェルスクエア学芸大学    東京都目黒区鷹番３   
2              ウェルスクエア学芸大学    東京都目黒区鷹番３   
3              ウェルスクエア学芸大学    東京都目黒区鷹番３   
4              ウェルスクエア学芸大学    東京都目黒区鷹番３   
6              ウェルスクエア学芸大学    東京都目黒区鷹番３   
7              セザール大岡山ガーデン   東京都目黒区大岡山１   
8      東急目黒線 大岡山駅 4階建 築28年   東京都目黒区大岡山１   
9     東急東横線 学芸大学駅 3階建 築31年  東京都目黒区目黒本町１   
11                セザール学芸大学    東京都目黒区鷹番２   
12    東急東横線 学芸大学駅 6階建 築48年    東京都目黒区鷹番２   
14                   ネスト鷹番    東京都目黒区鷹番１   
15  東急田園都市線 池尻大橋駅 3階建 築18年   東京都目黒区青葉台３   
16  東急田園都市線 池尻大橋駅 3階建 築18年   東京都目黒区青葉台３   
17                 ジュネス青葉台   東京都目黒区青葉台３   
18                 ジュネス青葉台   東京都目黒区青葉台３   
19                 ジュネス青葉台   東京都目黒区青葉台３   
21      東急東横線 学芸大学駅 5階建 新築    東京都目黒区鷹番３   
22      東急東横線 学芸大学駅 5階建 新築    東京都目黒区鷹番３   
23      東急東横線 学芸大学駅 5階建 新築    東京都目黒区鷹番３   
24      東急東横線 学芸大学駅 5階建 新築    東京都目黒区鷹番３   
25      東急東横線 学芸大学駅 5階建 新築    東京都目黒区鷹番３   
28     東急東横

In [41]:
# 重複した行を削除
d = d.drop_duplicates(subset=['住所', '築年数', '階数'], keep='first')

In [42]:
d.head()

,物件名,住所,アクセス,築年数,階数,賃料,管理費,敷金,礼金,間取り,面積
0,エルファーロ大岡山II,東京都目黒区大岡山１,\n大岡山駅 歩7分\n緑が丘駅 歩14分\n洗足駅 歩14分\n,\n新築\n4階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t3,19.7万円,10000円,-,-,2LDK,45.02m2
1,ウェルスクエア学芸大学,東京都目黒区鷹番３,\n学芸大学駅 歩2分\n祐天寺駅 歩17分\n都立大学駅 歩22分\n,\n新築\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1,18.6万円,12000円,18.6万円,18.6万円,2LDK,40.8m2
2,ウェルスクエア学芸大学,東京都目黒区鷹番３,\n学芸大学駅 歩2分\n祐天寺駅 歩17分\n都立大学駅 歩22分\n,\n新築\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2,19.4万円,12000円,19.4万円,19.4万円,2LDK,40.8m2
3,ウェルスクエア学芸大学,東京都目黒区鷹番３,\n学芸大学駅 歩2分\n祐天寺駅 歩17分\n都立大学駅 歩22分\n,\n新築\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t3,19.6万円,12000円,19.6万円,19.6万円,2LDK,40.8m2
5,ウェルスクエア学芸大学,東京都目黒区鷹番３,\n学芸大学駅 歩2分\n祐天寺駅 歩17分\n都立大学駅 歩22分\n,\n新築\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t4,19.8万円,12000円,19.8万円,19.8万円,2LDK,40.8m2


In [43]:
# dfをcsvに出力
d.to_csv('suumo.csv', index=None, encoding='utf-8-sig')